<a href="https://colab.research.google.com/github/apchavezr/-Analisis_Grandes_Volumenes_Datos/blob/main/Ejercicio_PySpark_AnalisisVentas_Explicado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧪 Ejercicio práctico: Análisis de ventas por región usando PySpark
Este notebook permite analizar el comportamiento de ventas por región y canal de venta. Se utilizará PySpark para el procesamiento de datos distribuido.

In [1]:
# Paso 1: Inicializar sesión de Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("AnalisisVentasRegionales") \
    .getOrCreate()

In [2]:
# Paso 2: Crear datos de ejemplo
data = [
    ("Región Andina", "Online", 2023, 5000),
    ("Región Andina", "Presencial", 2023, 7000),
    ("Región Caribe", "Online", 2023, 3000),
    ("Región Caribe", "Presencial", 2023, 4500),
    ("Región Pacífica", "Online", 2023, 2000),
    ("Región Pacífica", "Presencial", 2023, 3500),
    ("Región Andina", "Online", 2024, 5500),
    ("Región Andina", "Presencial", 2024, 7500),
    ("Región Caribe", "Online", 2024, 3200),
    ("Región Caribe", "Presencial", 2024, 4700),
    ("Región Pacífica", "Online", 2024, 2500),
    ("Región Pacífica", "Presencial", 2024, 3700),
]

columnas = ["region", "canal_venta", "anio", "ventas"]
df = spark.createDataFrame(data, columnas)
df.show()

+---------------+-----------+----+------+
|         region|canal_venta|anio|ventas|
+---------------+-----------+----+------+
|  Región Andina|     Online|2023|  5000|
|  Región Andina| Presencial|2023|  7000|
|  Región Caribe|     Online|2023|  3000|
|  Región Caribe| Presencial|2023|  4500|
|Región Pacífica|     Online|2023|  2000|
|Región Pacífica| Presencial|2023|  3500|
|  Región Andina|     Online|2024|  5500|
|  Región Andina| Presencial|2024|  7500|
|  Región Caribe|     Online|2024|  3200|
|  Región Caribe| Presencial|2024|  4700|
|Región Pacífica|     Online|2024|  2500|
|Región Pacífica| Presencial|2024|  3700|
+---------------+-----------+----+------+



In [3]:
# Paso 3: Cálculo del promedio de ventas por región y canal
from pyspark.sql.functions import avg, col, when

promedios = df.groupBy("region", "canal_venta") \
    .agg(avg("ventas").alias("promedio_ventas"))
promedios.show()

+---------------+-----------+---------------+
|         region|canal_venta|promedio_ventas|
+---------------+-----------+---------------+
|Región Pacífica| Presencial|         3600.0|
|  Región Caribe| Presencial|         4600.0|
|  Región Andina|     Online|         5250.0|
|Región Pacífica|     Online|         2250.0|
|  Región Andina| Presencial|         7250.0|
|  Región Caribe|     Online|         3100.0|
+---------------+-----------+---------------+



In [4]:
# Paso 4: Clasificación según promedio de ventas
resultado = promedios.withColumn(
    "categoria",
    when(col("promedio_ventas") >= 5000, "Alta").otherwise("Media o Baja")
)
resultado.show()

+---------------+-----------+---------------+------------+
|         region|canal_venta|promedio_ventas|   categoria|
+---------------+-----------+---------------+------------+
|Región Pacífica| Presencial|         3600.0|Media o Baja|
|  Región Caribe| Presencial|         4600.0|Media o Baja|
|  Región Andina|     Online|         5250.0|        Alta|
|Región Pacífica|     Online|         2250.0|Media o Baja|
|  Región Andina| Presencial|         7250.0|        Alta|
|  Región Caribe|     Online|         3100.0|Media o Baja|
+---------------+-----------+---------------+------------+



In [5]:
# Paso 5: Exportar a Pandas (opcional para visualización si usa Colab)
resultado.toPandas()

,region,canal_venta,promedio_ventas,categoria
0,Región Pacífica,Presencial,3600.0,Media o Baja
1,Región Caribe,Presencial,4600.0,Media o Baja
2,Región Andina,Online,5250.0,Alta
3,Región Pacífica,Online,2250.0,Media o Baja
4,Región Andina,Presencial,7250.0,Alta
5,Región Caribe,Online,3100.0,Media o Baja


## 💡 Preguntas guía
1. ¿Qué regiones muestran un comportamiento de ventas más alto en el canal online?
2. ¿Cómo varían las ventas entre los años 2023 y 2024?
3. ¿Qué transformaciones adicionales se podrían aplicar al DataFrame para segmentar por trimestre o producto?

## 🔍 Desarrollo de preguntas guía

### 1. ¿Qué regiones muestran un comportamiento de ventas más alto en el canal online?
Vamos a filtrar los datos para el canal de venta 'Online' y luego ordenar las regiones por su promedio de ventas de forma descendente.

In [6]:
online = resultado.filter(resultado.canal_venta == "Online")
online.orderBy("promedio_ventas", ascending=False).show()

+---------------+-----------+---------------+------------+
|         region|canal_venta|promedio_ventas|   categoria|
+---------------+-----------+---------------+------------+
|  Región Andina|     Online|         5250.0|        Alta|
|  Región Caribe|     Online|         3100.0|Media o Baja|
|Región Pacífica|     Online|         2250.0|Media o Baja|
+---------------+-----------+---------------+------------+



**Explicación:** La tabla anterior muestra las regiones que operan en el canal Online, ordenadas por su rendimiento. Se observa que la Región Andina tiene el mayor promedio de ventas en línea, lo cual puede deberse a una mayor penetración tecnológica o estrategias de comercio digital más desarrolladas.

### 2. ¿Cómo varían las ventas entre los años 2023 y 2024?
Ahora vamos a agrupar por año y calcular el promedio total de ventas para observar la evolución anual.

In [7]:
df.groupBy("anio").agg(avg("ventas").alias("promedio_anual")).orderBy("anio").show()

+----+-----------------+
|anio|   promedio_anual|
+----+-----------------+
|2023|4166.666666666667|
|2024|4516.666666666667|
+----+-----------------+



**Explicación:** Este análisis permite identificar tendencias interanuales. Un aumento en el promedio de ventas puede reflejar un mejor desempeño comercial, mayor cobertura del sistema o estrategias de venta más efectivas.

### 3. ¿Qué transformaciones adicionales se podrían aplicar al DataFrame para segmentar por trimestre o producto?
Aunque el conjunto de datos no tiene productos ni fechas exactas, podemos simular una transformación para segmentar por trimestre si tuviéramos una columna de fecha.

In [ ]:
# Simulación: si existiera una columna 'fecha', podríamos hacer esto:
# from pyspark.sql.functions import quarter, to_date
# df = df.withColumn("fecha", to_date(df.fecha, "yyyy-MM-dd"))
# df = df.withColumn("trimestre", quarter("fecha"))

**Explicación:** Estas transformaciones son comunes en análisis de ventas para estudiar estacionalidad o comportamientos periódicos. En proyectos reales, también es común incorporar variables como categoría de producto, promociones, y canales cruzados.